## Lab 6: Scikit-Learn

**Goal:** In this notebook we are going to start applying machine learning methods to train statistical models and explore new methods of inference and prediction. We will focus on Supervised Learning, both regression (learn continuous labels) and classification (learn discrete labels). In particular, we will introduce the Python library Scikit-learn and play with:
- Linear and Polynomial regression
- Naive Bayes

The goals of this notebook are to test the application of Regression and Classification methods to structured and unstructured datasets; to apply model validation methods; and to perform feature engineering on unstructured text. 

The practical aspects of this Lab will be based on the **Chapter 5** (up to section *In Depth: Linear Regression*) of the [*Python Data Science Handbook*](https://jakevdp.github.io/PythonDataScienceHandbook/).

### Part I: Linear and polynomial regression

Here we will apply Scikit learn to create a Linear Regression model. Similar to other methods that use Estimator API of Scikit learn library, training and applying Linear Regression is a 5-Step process:

1. Select model (Linear Regression, in this case) and import it
2. Select model hyperparameters
3. Arrange data in feature matrix (or vector if just 1 feature) and Target array
4. Fit model to data
5. Apply model to inference and prediction problems

We will apply Linear Regression to better **understand 1) the relationship between GDP and Life Expectancy and 2) the relationship between Life expectancy and *happiness* in a specific year (2016)**.

First, we will load and vizualize the required data:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data
happinessdataframe = pd.read_excel('happiness.xls', index_col=[0,1])

x = happinessdataframe.dropna().loc(axis=0)[:,2016][["Log GDP per capita","Healthy life expectancy at birth"]]

x.plot.scatter(x="Log GDP per capita", y="Healthy life expectancy at birth",c='black')
plt.show()

Below you have an example of ordinary least squares Linear Regression.

LinearRegression fits a linear model with coefficients $\vec{w} = (w_0, w_1,..., w_p)$ to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

When $p=1$ (that is, a feature space with 1 dimension), LinearRegression fits a straight line to the data. Such line (as any other straight line) can be defined by two parameters: intercept ($w_0$) and slope ($w_1$).

In [ ]:
# 5 Steps to use the Scikit-learn Estimator API:

# 1. Select model and import it
from sklearn.linear_model import LinearRegression

# 2. Select model hyperparameters
model = LinearRegression(fit_intercept=True) # select model hyperparameters

# 3. Arrange data in feature matrix (or vector if just 1 feature) and Target array
X = x["Log GDP per capita"].values
Y = x["Healthy life expectancy at birth"].values

# 4. Fit model to data
model.fit(X=X[:, np.newaxis], y=Y) # / np.newaxis increases dimension of array by 1

# 5. Apply model (in this case, visualize the regression line)
xfit = np.linspace(min(X), max(X), num=2) # / linspace returns matrix with evenly spaced numbers
yfit = model.predict(X=xfit[:, np.newaxis])

# Plot
plt.scatter(X, Y, c='black')
plt.plot(xfit, yfit, c='red');

### Exercise 1

What is the slope (or coefficient) and intercept (or bias) of the model created? What is the slope telling us about the relationship between GDP and life expectancy?

In [ ]:
# Your code here

### Exercise 2

Can you write down an expression for the straight red line in the previous plot, using the values of intercept_ and coef_ that were just printed? To confirm your solution, do a line plot where you overlap the red plot in the figure below (red) with another plot with the straight line equation you derived.

In [ ]:
# Your code here

Linear regression can also be applied after expanding the feature space to include non-linear terms. This is convenient to capture non-linear relationships in data; notice that the number of coefficients will also increase. 

Let us now inspect the relationship between Life expectancy and Happiness. First we plot the data:

In [ ]:
x = happinessdataframe.dropna().loc(axis=0)[:,2016][["Healthy life expectancy at birth","Life Ladder"]]
x.plot.scatter(x="Healthy life expectancy at birth", y="Life Ladder",c='black')
plt.show()

Note that now the relationship between the predictor and response of interest doesn't appear to be linear. 

We now intend to expand the feature space to include higher order terms using, as basis function, a polynomial function of degree 2.

You can notice, below, a pipeline. To gain some insight on what is the meaning of a pipeline, please check the PDSH book, [here](https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html) (feature pipelines).

### Exercise 3

Fill the missing lines below to fit the model to data and apply the model.

In [ ]:
# 1. Select model and import it
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# 2. Select model hyperparameters (here, we will use a polynomial basis function, of degree 2)
polymodel = make_pipeline(PolynomialFeatures(2), LinearRegression(fit_intercept=True))

# 3. Arrange data in feature matrix (or vector if just 1 feature) and Target array
x = happinessdataframe.dropna().loc(axis=0)[:,2016][["Healthy life expectancy at birth","Life Ladder"]]

X = x["Healthy life expectancy at birth"].values
Y = x["Life Ladder"].values

# 4. Fit model to data
# Your code here

# 5. Apply model
xfit = np.linspace(min(X), max(X), 100)
# Your code here

# Plot
fig, ax = plt.subplots()
ax.scatter(X, Y, c='black')
ax.set_xlabel("Healthy life expectancy at birth")
ax.set_ylabel("Life Ladder")
ax.plot(xfit, yfit, c='red');
plt.show()

Should we use Linear or Polynomial regression to model the relationship between Life expectancy and Happiness? 

Let us use historic data (2016) to fit a model and use it to predict Happiness (Life Ladder) in 2017. We'll use a Linear or Polynomial regression. To quantify which model predicts better the relationship between Life expectancy and Happiness in 2017 we will compute the mean square error (MSE) and the coefficient of determination (R2) score between the real data in 2017 and our prediction.

Notice that now we are considering a training (2016 data) and testing set (2017 data) — when doing hyperparameter tuning it is also common to use a validation set (more details below).

In [ ]:
# Calculate MSE and R2

from sklearn.metrics import mean_squared_error, r2_score

# Split the data into training/testing sets

# 2016 will be our TRAIN set
x = happinessdataframe.dropna().loc(axis=0)[:,2016][["Healthy life expectancy at birth", "Life Ladder"]]

Xtrain = x["Healthy life expectancy at birth"].values
Ytrain = x["Life Ladder"].values

polymodel = make_pipeline(PolynomialFeatures(1), LinearRegression(fit_intercept=True))
polymodel.fit(Xtrain[:, np.newaxis], Ytrain)

# 2017 will be our TEST set
x = happinessdataframe.dropna().loc(axis=0)[:,2017][["Healthy life expectancy at birth","Life Ladder"]]

Xtest = x["Healthy life expectancy at birth"].values
Ytest = x["Life Ladder"].values

YPredictionTest = polymodel.predict(Xtest[:, np.newaxis])

# The mean squared error
print("Mean squared error - Test: %.2f" % mean_squared_error(Ytest, YPredictionTest))
# Explained variance score: 1 is perfect prediction
print('Variance score - Test: %.2f' % r2_score(Ytest, YPredictionTest))

### Exercise 4

The previous evaluation metrics refer to a Linear regression (note the *PolynomialFeatures(1)*). What are the mean squared error and coefficient of determination (R2) if we apply a polynomial regression of degree 2? Does that mean that we should opt for a linear or polynomial regression, to model the relationship between Life expectancy and Happiness?**

*Expected output:* 

MSE and R2 (aka coefficient of determination or variance score) associated with the polynomial regression with degree 2.

In [ ]:
# Your code here

### Exercise 5

We could use data before 2016 to train our model. Will that improve test error and variance score? Check that..

In [ ]:
# Your code here

### Exercise 6

1) Create a regression model to capture the relationship between 'Freedom to make life choices' and 'Perception of corruption' in 2017. 2) Do a scatter plot with the corresponding data and regression line. Which order of basis function better capture the relationship between Freedom to make life choices and Perception of corruption in 2017?

*Expected output:*

You can check visually which order better captures the relationship in data (try order 1 to, say, 5); alternativelly, calculate the MSE and R2 associated with each order.

*Recall that you change the order of the polynomial basis function by changing argument X in PolynomialFeatures(X)*

In [ ]:
# Your code here

### Exercise 7

Why do you think we need a test and validation set? Why not using just a single set to do validation and tests?


In [ ]:
# Your code here

### Part II: Classification (Naive Bayes)

Previously we applied Regression methods given a structured dataset. We will now apply classification methods to an unstructured (text) dataset. Notice that instead of having continuous targets we will have discrete categorical targets. 

We will use as example the (quite famous) [Newsgroup](http://qwone.com/~jason/20Newsgroups/) dataset. The [20 Newsgroups collection](http://qwone.com/~jason/20Newsgroups/) has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering. This example will be based on the [Scikit documentation](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html). 

The main goals of the following section are to have you experiment your skills with unstructured data (text), play with feature engineering in text and apply a classifcation algorithm (Naive Bayes). 

In [ ]:
# Import newsgroup dataset from sklearn datasets
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

# Select the categories of interest
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

# Sample a training set
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

To create a feature vector from unstructured text, we will encode *text* as *numbers* by performing simple word counting. We will assign each word to a numerical *id* and count the occurrence of each word. We can easily perform such task by using the *CountVectorizer* class included in Scikit learn:

In [ ]:
# Feature Engineering:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts

The result above is a sparse matrix. It is efficient to represent the number of times each word appears as a sparse matrix as most of the entries in the matrix will be 0.

In [ ]:
# let us check the contents of the word matrix:
pd.DataFrame(X_train_counts.toarray(), columns=count_vect.get_feature_names()).head()

In [ ]:
# We now apply the (Multinomial) Naive Bayes; 

# Remember the 5 steps we need to apply...

# 1. Select model and import it
from sklearn.naive_bayes import MultinomialNB

# 2. Select model hyperparameters; alpha is a smoothing parameter
model = MultinomialNB(alpha=10)

# 3. Arrange data in feature matrix / perform feature engineering
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
Y = twenty_train.target

# 4. Fit model to data
clf = model.fit(X_train_counts, Y)

# 5. Apply model to new examples
docs_new = ['God is love', 'OpenGL on the GPU is fast','Help with printer','My knee hurts']

X_new_counts = count_vect.transform(docs_new)

predicted = clf.predict(X_new_counts)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

We created a model trained on previous examples of text and categories and applied it to new example. 

In general, how *good* is our model? Here we will validate the model we created and understand how to tune it to perform better.

We will use the **TfidfTransformer**: this will transform a count matrix to a normalized tf or tf-idf representation; *Tf* means term-frequency while *tf-idf* means term-frequency times inverse document-frequency. This is a common term weighting scheme in information retrieval, that has also found good use in document classification. The goal of using tf-idf instead of the raw frequencies of occurrence of a token in a given document (as in the previous example) is to scale down the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus. If needed, more info [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html). 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB(alpha=1))])

text_clf.fit(twenty_train.data, twenty_train.target)

# create a test set -> this is a set different than the train set
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)

#in out many examples if our model guessing the right post category?
np.mean(predicted == twenty_test.target)

In the example above we are computing model accuracy, as the fraction of correctly predicted text classes.

In general, we can distinguish between the number of examples belonging to each class that that are correctly or incorrectly classified; this leads to the so-called *confusion matrix*. Each row of the matrix represents the instances in an actual class (true value) while each column represents the instances in a predicted class (predicted value). The name stems from the fact that it makes it easy to see whether the system is confusing two classes (i.e. commonly mislabeling one as another).

In [ ]:
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(twenty_test.target, predicted)

print(mat)

fig, ax = plt.subplots()
ax.imshow(mat, cmap='viridis', interpolation='nearest')
ax.set_xlabel('predicted value')
ax.set_ylabel('true value');
ax.set_xticks([0,1,2,3])
ax.set_yticks([0,1,2,3])
ax.set_xticklabels(categories, rotation='vertical')
ax.set_yticklabels(categories)

# Loop over data to create text annotations.
for i in range(len(mat)):
    for j in range(len(mat)):
        text = ax.text(j, i, mat[i, j],
                       ha="center", va="center", color="w")
plt.show()

### Exercise 8

- a) How many times are texts classified as med?

- b) How many times are texts not classified as med?

- c) How many times are texts belonging to category med correctly classified? (True Positives - TP)

- d) How many times are texts belonging to category med wrongly classified? (False Negative - FN)

- e) How many times are texts not belonging to category med correctly classified as not being med? (True Negatives - TN)

- f) What is the sensitivity of this model associated with class med? (Sensitivity = TP/(TP+FN))

- Q9.7: What is the specificity of this model associated with class christian? (Specificity = TN/(TN+FP))

*Expected output:* 

One numerical value for each question.

Your answers here